# Data Analysis

# Clinic1: Exploratory Data Analysis

In this clinic, first we will:

* Cover a short intro to pandas series and dataframes
* Learn how to read in and write files

Then we will start learning the following that you will optimally practice during the clinic deliverable tasks :

* Load files and systematically check their integrity
* Parse columns in the dataframe to create new dataframe columns
* Create and interpret visualizations to explore the data set

# 1. The Pandas Library: Series

Pandas is a popular library for manipulating vectors, tables, and time series. We will frequently use Pandas data structures instead of the built-in python data structures, as they provide much richer functionality. Also, Pandas is **fast**, which makes working with large datasets easier.  Check out the official pandas website at [http://pandas.pydata.org/](http://pandas.pydata.org/).

This tutorial is partially based on the [excellent book by Matt Harrison](https://www.amazon.com/Learning-Pandas-Library-Analysis-Visualization-ebook/dp/B01GIE03GW/).

When you work with Pandas, it's handy to have a [cheat sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) lying around. 

Pandas provides three data structures: 

 * the **series**, which represents a single column of data similar to a python list
 * the **data frame**, which represents multiple series of data
 * the **panel**, which represents multiple data frames
 
We'll mostly work with series and data frames and largely ignore panels. We will start with the series. 

Pandas should already be part of your anaconda installation. If not, simply run:

```
$ conda install pandas
```

To make pandas available, we'll import the module into this notebook. It is customary to import pandas as `pd`:

In [1]:
import pandas as pd

Series are the most fundamental data structure in pandas. Let's create two simple series based on an arrays:

In [2]:
bands = pd.Series(["Stones", "Beatles", "Zeppelin", "Pink Floyd"])
bands

0        Stones
1       Beatles
2      Zeppelin
3    Pink Floyd
dtype: object

In [3]:
founded = pd.Series([1962, 1960, 1968, 1965])
founded

0    1962
1    1960
2    1968
3    1965
dtype: int64

When we output these objects we can see an index, also called an axis, which by default is an integer sequence starting at 0, and the associated values. 

| Index | Value | 
| - | - |
| 0  |        Stones
|1   |    Beatles
|2  |    Zeppelin
|3 |    Pink Floyd

Pandas also tells us the data type of the values, `object` for the first series – in this case, this is a string, `int64` (a 64-bit integer) for the second.

Notice that `int64` is not a Python datatype, but a C integer of 64 bit length – which, unlike Python integers – can overflow!

We can also use other data types as indices, in which case the series behaves a lot like a dictionary:

In [4]:
# the data is the first parameter, the index is given by the index keyword
bands_founded = pd.Series([1962, 1960, 1968, 1965, 2012],
                          index=["Stones", "Beatles", "Zeppelin", "Pink Floyd", "Pink Floyd"], 
                          name="Bands founded")
bands_founded

Stones        1962
Beatles       1960
Zeppelin      1968
Pink Floyd    1965
Pink Floyd    2012
Name: Bands founded, dtype: int64

| Index | Value | 
| - | - |
| Stones     |    1962
| Beatles    |    1960
| Zeppelin     |  1968
| Pink Floyd |    1965
| Pink Floyd |    2012

Here we see something interesting: We've used the same index (Pink Floyd) twice, once for the original founding of the band, and once for the re-union starting in 2012. Also, the order of the entries is preserved. 

A series is both, a list and a dictionary! 

We can access the values of an array by printing the member `values`.

In [5]:
bands.values

array(['Stones', 'Beatles', 'Zeppelin', 'Pink Floyd'], dtype=object)

And we can look at how the index is composed:

In [6]:
bands.index

RangeIndex(start=0, stop=4, step=1)

What we see here is that this isn't an explicit list, but rather a set of rules, similar to the ranges we've already worked with. 

Let's compare this to the index where we used explicit labels:

In [7]:
bands_founded.index

Index(['Stones', 'Beatles', 'Zeppelin', 'Pink Floyd', 'Pink Floyd'], dtype='object')

We can access individual entries as we'd access an array or a dictionary:

In [ ]:
bands[0]

In [8]:
bands_founded["Beatles"]

1960

There is also a method for looking up a value:

In [9]:
bands_founded.get("Stones")

1962

Note that these access methods are as fast as a dictionary lookup, and much faster than a lookup in a list.

That works also with arrays of labels, in which case the return type is a series, not a single value.

In [10]:
bands_founded.get(["Stones", "Beatles"])

Stones     1962
Beatles    1960
Name: Bands founded, dtype: int64

Notice that when we access data with multiple indices, we don't get a simple datatype, as in the above cases, but instead get another series back:

In [11]:
bands_founded["Pink Floyd"]

Pink Floyd    1965
Pink Floyd    2012
Name: Bands founded, dtype: int64

Series also have indexers for label-based access: [`loc`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.loc.html)

In [12]:
# And one more way for looking up a value:
bands_founded.loc["Stones"]
# this is equivalent to 
# bands_founded["Stones"]

1962

Related to the `loc` indexer is the [`iloc`](http://pandas.pydata.org/pandas-docs/version/0.17.0/generated/pandas.DataFrame.iloc.html) indexer. However, instead of operating on our label index, `iloc` operates purely on position: 

In [13]:
bands_founded.iloc[1]

1960

There is also an `ix` indexer, which is deprecated and should not be used.

These ways of accessing slices of a dataset (`loc`, `iloc`), will make more sense when we use dataframes instead of series – in dataframes, `loc` and `iloc` operate on the rows, whereas square brackets operate on the columns.

### 1.1 Iterating

Iteration works as you would expect: 

In [14]:
for band in bands:
    print(band)

Stones
Beatles
Zeppelin
Pink Floyd


In [15]:
for band, founded in bands_founded.items():
    print(band + ", " + str(founded))

Stones, 1962
Beatles, 1960
Zeppelin, 1968
Pink Floyd, 1965
Pink Floyd, 2012


### 1.2 Updating
Updating works largely as expected, however, you have to be careful when updating series with duplicate indices:

In [16]:
bands[2] = "The Doors"
bands

0        Stones
1       Beatles
2     The Doors
3    Pink Floyd
dtype: object

We can add a new item by direclty assigning it to a new index.

In [17]:
bands[4] = "Zeppelin"
bands

0        Stones
1       Beatles
2     The Doors
3    Pink Floyd
4      Zeppelin
dtype: object

Note that the indices don't have to be sequential.

In [18]:
bands[17] = "The Who"
bands

0         Stones
1        Beatles
2      The Doors
3     Pink Floyd
4       Zeppelin
17       The Who
dtype: object

When we update based on an index that occurs more than once, all instances are updated:

In [19]:
bands_founded["Pink Floyd"] = 2015
bands_founded

Stones        1962
Beatles       1960
Zeppelin      1968
Pink Floyd    2015
Pink Floyd    2015
Name: Bands founded, dtype: int64

A way to update a specific entry when an index is used multiple time is to use the `iloc` indexer. We can use the `iloc` array to set values based purely on position. However, all of this is rather ugly.

In [20]:
bands_founded.iloc[3] = 1965
bands_founded

Stones        1962
Beatles       1960
Zeppelin      1968
Pink Floyd    1965
Pink Floyd    2015
Name: Bands founded, dtype: int64

### 1.3 Deleting 

Deleting is rarely done with pandas data structures, instead filters and masks are used. It's possible based on indices:

In [21]:
del bands_founded["Stones"]
bands_founded

Beatles       1960
Zeppelin      1968
Pink Floyd    1965
Pink Floyd    2015
Name: Bands founded, dtype: int64

### 1.4 Indexing and slicing

Indexing and slicing works largely like in normal python, but instead of just directly using the bracket notations, it is recommended to use `iloc` for indexing by position and `loc` for indexing by labelled indices. 

In [22]:
# slicing by position
bands_founded.iloc[1:3]

Zeppelin      1968
Pink Floyd    1965
Name: Bands founded, dtype: int64

When slicing by labelled index, the last value specified is *included*, which differs from regular Python slicing behavior.

In [23]:
# slicing by index
bands_founded.loc["Zeppelin" : "Pink Floyd"]

Zeppelin      1968
Pink Floyd    1965
Pink Floyd    2015
Name: Bands founded, dtype: int64

In [24]:
# Note that index 17 is included
bands.loc[1:17]

1        Beatles
2      The Doors
3     Pink Floyd
4       Zeppelin
17       The Who
dtype: object

Again, for series (not for data frames), `loc` and just using bracket notation is identical: 

In [25]:
bands[2:17]

2      The Doors
3     Pink Floyd
4       Zeppelin
17       The Who
dtype: object

Both, `iloc` and `loc` can be used with arrays, which isn't possible in vanilla Python:

In [26]:
bands_founded.iloc[[0,3]]

Beatles       1960
Pink Floyd    2015
Name: Bands founded, dtype: int64

In [27]:
bands_founded.loc[["Beatles", "Pink Floyd"]]

Beatles       1960
Pink Floyd    1965
Pink Floyd    2015
Name: Bands founded, dtype: int64

And, all these variants can also be used with boolean arrays, which we will soon find out to be very helpful:

In [28]:
bands_founded

Beatles       1960
Zeppelin      1968
Pink Floyd    1965
Pink Floyd    2015
Name: Bands founded, dtype: int64

In [29]:
bands_founded.loc[[True, False, False, True]]

Beatles       1960
Pink Floyd    2015
Name: Bands founded, dtype: int64

### 1.5 Masking and Filtering

With pandas we can create boolean arrays that we can use to mask and filter a dataset. In the following expression, we'll create a new array that has "True" for every band formed after 1964:

In [30]:
mask = bands_founded > 1964
mask

Beatles       False
Zeppelin       True
Pink Floyd     True
Pink Floyd     True
Name: Bands founded, dtype: bool

This uses a technique called **broadcasting**. We can use broadcasting with various operations:

In [31]:
# Not particularly useful for this dataset..
founding_months = bands_founded * 12
founding_months

Beatles       23520
Zeppelin      23616
Pink Floyd    23580
Pink Floyd    24180
Name: Bands founded, dtype: int64

We can use a boolean mask to filter a series, as we've seen before:

In [32]:
# applying the mask to the original array
# note that almost all of those operations return a new copy and don't modify in place
bands_founded[mask]

Zeppelin      1968
Pink Floyd    1965
Pink Floyd    2015
Name: Bands founded, dtype: int64

The short form here would be:

In [33]:
bands_founded[bands_founded > 1967]

Zeppelin      1968
Pink Floyd    2015
Name: Bands founded, dtype: int64

### 1.6 Exploring a Series

There are various way we can explore a series. We can count the number of non-null values: 

In [35]:
numbers = pd.Series([1962, 1960, 1968, 1965, 2012, None, 2016])
numbers.count()

6

In [36]:
numbers

0    1962.0
1    1960.0
2    1968.0
3    1965.0
4    2012.0
5       NaN
6    2016.0
dtype: float64

We can get the sum, mean, median of a series:

In [37]:
numbers.sum()

11883.0

In [38]:
numbers.mean()

1980.5

In [39]:
numbers.median()

1966.5

We can also get an overview of the statistical properties of a series: 

In [40]:
numbers.describe()

count       6.000000
mean     1980.500000
std        26.120873
min      1960.000000
25%      1962.750000
50%      1966.500000
75%      2001.000000
max      2016.000000
dtype: float64

Note that None/NaN values are ignored here. We can drop all NaN values if we desire:

In [41]:
numbers = numbers.dropna()
numbers

0    1962.0
1    1960.0
2    1968.0
3    1965.0
4    2012.0
6    2016.0
dtype: float64

This works also for non-numerical data. Of course, we get different measures:

In [42]:
bands.describe()

count          6
unique         6
top       Stones
freq           1
dtype: object

Other useful methods are asking for a specific quantile, the minimum, the maximum, etc. 

In [43]:
numbers.quantile(0.25)

1962.75

In [44]:
numbers.max()

2016.0

In [45]:
numbers.min()

1960.0

### 1.7 Sorting 

We can sort a series:

In [46]:
numbers.sort_values()

1    1960.0
0    1962.0
3    1965.0
2    1968.0
4    2012.0
6    2016.0
dtype: float64

And make the sorting descending: 

In [53]:
sorted_numbers = numbers.sort_values(ascending=False)
sorted_numbers

6    2016.0
4    2012.0
2    1968.0
3    1965.0
0    1962.0
1    1960.0
dtype: float64

Note that the indices remain the same! We can **reset the indices**:

In [54]:
# If we don't specify drop to be true, the previous indices are preserved in a separte column
sorted_numbers = sorted_numbers.reset_index(drop=True)
sorted_numbers

0    2016.0
1    2012.0
2    1968.0
3    1965.0
4    1962.0
5    1960.0
dtype: float64

We can also sort by the index:

In [55]:
# mix up the indices first
new_sorted_numbers = numbers.sort_values()
print(new_sorted_numbers)
print(new_sorted_numbers.sort_index())

1    1960.0
0    1962.0
3    1965.0
2    1968.0
4    2012.0
6    2016.0
dtype: float64
0    1962.0
1    1960.0
2    1968.0
3    1965.0
4    2012.0
6    2016.0
dtype: float64


### 1.8 Applying a Function

Often, we will want to apply a function to all values of a Series. We can do that with the [`map()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html) function:

In [65]:
import datetime

# Convert an integer year into a date, assuming Jan 1 as day and month.
def to_date(year):
    return datetime.date(int(year), 1, 1)
    
new_sorted_numbers.map(to_date)

1    1960-01-01
0    1962-01-01
3    1965-01-01
2    1968-01-01
4    2012-01-01
6    2016-01-01
dtype: object

This is an incredibly powerful concept that you can use to modify series in sophisticated ways, similar to list comprehension. 

Another way to use the map function is to pass in a dictionary that is then applied to matching objects: 

In [69]:
hghgh = new_sorted_numbers.map({1965:1945, 2012:1999, 1968:"What"})

hghgh.iloc[3]



'What'

 We've only covered a small part of the features of Series here. Make sure to also check out resources such as the [10 minutes to pandas guide](http://pandas.pydata.org/pandas-docs/stable/10min.html).

# 2. Reading Data

Up to now, we've mainly used data that we've specified directly in code. This is, of course, not particularly scalable. We want to load data from files and eventually also connect to databases and APIs. 

Data is often stored in structured file formats, such as CSV, JSON, or XML. We'll encounter all of these file formats in this class.

JSON

```json
{
    "fruit": "Apple",
    "size": "Large",
    "color": "Red"
}
```

XML
```xml
<note>
<to>Students</to>
<from>Prof</from>
<heading>Reminder</heading>
<body>HW2 due this Friday at 11:59pm!</body>
</note>
```

The simplest form is a CSV (Comma Separated Values) file. CSV isn't a formal file format, rather it's a table represented as a text file where the cells are separated by a delimiter. Commonly, the first row represents the header. A delimiter can be a tab character, a semicolon, a colon, etc. 

Many CSV files also have a special convention for dealing with text that could include the delimiter. The following text would be very hard to parse otherwise:
```
Artist, Album, Genre
Michael Jackson, Bad, Pop, Funk, Rock
``` 

Here, the album is of multiple genres which are separated by a comma. The comma, however, is also used to delimit the individual columns. 

To work around that, double-quotes are commonly used (though other escape characters are possible) to indicate that all the elements contained within the quotes are not meant to be delimiters:

```
Artist, Album, Genre
Michael Jackson, Bad, "Pop, Funk, Rock"
``` 

Of course, that's problematic if your text contains double-quotes.

Now, it is clear that `Pop, Funk, Rock` should belong in a single cell. 

We've prepared a dataset based on Wikipedia's [list of best-selling albums](https://en.wikipedia.org/wiki/List_of_best-selling_albums) in the file [hit_albums.csv](./hit_albums.csv). 

Here is what the first couple of lines look like:

```
Artist,Album,Released,Genre,"Certified sales (millions)",Claimed sales (millions)
Michael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65
AC/DC,Back in Black,1980,Hard rock,25.9,50
Pink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45
Whitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44
...
```

There are various ways of reading a CSV file. We'll first cover the basic read (and write) operations of Python, but will quickly move on to specific parsers for CSV files in Python and in Pandas. 

### 2.1 Basic File Operations

To read a file we first have to open it by specifying the file path, and specifying whether we want to read (r), write (w), both (r+), or append (a). 

In [79]:
albums_file = open('hit_albums.csv', 'r')

We can read a whole file at once. Notice that lines are terminated with a special character, a linefeed or newline character specified as `\n`.

In [80]:
content = albums_file.read()
content

'Artist,Album,Released,Genre,"Certified sales (millions)",Claimed sales (millions)\nMichael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65\nAC/DC,Back in Black,1980,Hard rock,25.9,50\nPink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45\nWhitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44\nMeat Loaf,Bat Out of Hell,1977,"Hard rock, progressive rock",20.6,43\nEagles,Their Greatest Hits (1971â€“1975),1976,"Rock, soft rock, folk rock",32.2,42\nBee Gees / Various artists,Saturday Night Fever,1977,Disco,19,40\nFleetwood Mac,Rumours,1977,Soft rock,27.9,40\nShania Twain,Come On Over,1997,"Country, pop",29.6,39\nLed Zeppelin,Led Zeppelin IV,1971,"Hard rock, heavy metal",29,37\nMichael Jackson,Bad,1987,"Pop, funk, rock",20.3,34\nAlanis Morissette,Jagged Little Pill,1995,Alternative rock,24.8,33\nCeline Dion,Falling into You,1996,"Pop, Soft rock",20.2,32\nThe Beatles,Sgt. Pepper\'s Lonely Hearts Club Band,1967,Rock,13.1,32\nEagles,Hotel Californ

If we print this instead, `\n` is translated into a newline: 

In [81]:
print(content)

Artist,Album,Released,Genre,"Certified sales (millions)",Claimed sales (millions)
Michael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65
AC/DC,Back in Black,1980,Hard rock,25.9,50
Pink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45
Whitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44
Meat Loaf,Bat Out of Hell,1977,"Hard rock, progressive rock",20.6,43
Eagles,Their Greatest Hits (1971â€“1975),1976,"Rock, soft rock, folk rock",32.2,42
Bee Gees / Various artists,Saturday Night Fever,1977,Disco,19,40
Fleetwood Mac,Rumours,1977,Soft rock,27.9,40
Shania Twain,Come On Over,1997,"Country, pop",29.6,39
Led Zeppelin,Led Zeppelin IV,1971,"Hard rock, heavy metal",29,37
Michael Jackson,Bad,1987,"Pop, funk, rock",20.3,34
Alanis Morissette,Jagged Little Pill,1995,Alternative rock,24.8,33
Celine Dion,Falling into You,1996,"Pop, Soft rock",20.2,32
The Beatles,Sgt. Pepper's Lonely Hearts Club Band,1967,Rock,13.1,32
Eagles,Hotel California,1976,"Rock, so

After reading a file, we have to manually close it again to release the OS resources:

In [82]:
albums_file.close()

As an alternative to reading the whole file, we can read each line separately:

In [83]:
albums_file = open('hit_albums.csv', 'r')
line1 = albums_file.readline();
print(line1)

Artist,Album,Released,Genre,"Certified sales (millions)",Claimed sales (millions)



We could now [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) the string based on the comma, to create a simple CSV parser:

In [84]:
line1.split(",")

['Artist',
 'Album',
 'Released',
 'Genre',
 '"Certified sales (millions)"',
 'Claimed sales (millions)\n']

After you have called  `readline()`, the next time you call it, it will read the next line. 

We can also treat albums_file as a list (or better, as an iterable data structure) and loop over the file and read the data into an array:

In [85]:
data = []
for line in albums_file:
    data.append(line.split(","))
    
# let's not forget to close the file:
albums_file.close()
data

[['Michael Jackson',
  'Thriller',
  '1982',
  '"Pop',
  ' rock',
  ' R&B"',
  '45.4',
  '65\n'],
 ['AC/DC', 'Back in Black', '1980', 'Hard rock', '25.9', '50\n'],
 ['Pink Floyd',
  'The Dark Side of the Moon',
  '1973',
  'Progressive rock',
  '22.7',
  '45\n'],
 ['Whitney Houston / Various artists',
  'The Bodyguard',
  '1992',
  '"Soundtrack/R&B',
  ' soul',
  ' pop"',
  '27.4',
  '44\n'],
 ['Meat Loaf',
  'Bat Out of Hell',
  '1977',
  '"Hard rock',
  ' progressive rock"',
  '20.6',
  '43\n'],
 ['Eagles',
  'Their Greatest Hits (1971â€“1975)',
  '1976',
  '"Rock',
  ' soft rock',
  ' folk rock"',
  '32.2',
  '42\n'],
 ['Bee Gees / Various artists',
  'Saturday Night Fever',
  '1977',
  'Disco',
  '19',
  '40\n'],
 ['Fleetwood Mac', 'Rumours', '1977', 'Soft rock', '27.9', '40\n'],
 ['Shania Twain', 'Come On Over', '1997', '"Country', ' pop"', '29.6', '39\n'],
 ['Led Zeppelin',
  'Led Zeppelin IV',
  '1971',
  '"Hard rock',
  ' heavy metal"',
  '29',
  '37\n'],
 ['Michael Jackson', '

We can now read individual cells or rows:

In [86]:
data[0]

['Michael Jackson',
 'Thriller',
 '1982',
 '"Pop',
 ' rock',
 ' R&B"',
 '45.4',
 '65\n']

In [87]:
data[0][1]

'Thriller'

As we can see, this **didn't take proper care of our double-quote escape of "Pop, rock, R&B"**. Also, numbers are still treated as strings and the newline character is also appended to the last cell. 

We could certainly improve our parser to handle these issues, but fortunately, there are existing methods to parse CSV files that make this easier.

### 2.2 Writing

We can write by opening a file using the `w` flag. Here we also use the [`with`](https://docs.python.org/3/reference/compound_stmts.html#the-with-statement) keyword, which takes care of closing the file for us, even if things go wrong (see [this blog post](https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/) for details):

In [88]:
with open('my_file.txt', 'w') as new_file:
    new_file.write("Hello World\nAre you still spinning?\n")

We can now check out this file by opening [my_file.txt](my_file.txt). Notice that the file is only guaranteed to be written if you actually close it (which, here, is take care of by the context manager invoked by the `with` statement). 

You can find more examples on basic file operations in the [Python Documentation](https://docs.python.org/3/tutorial/inputoutput.html).

### 2.3 Reading a CSV file with the CSV Library

We can use the CSV library to help with reading the data. It takes a `delimiter` and a `quotechar` as parameters; the latter is useful for our double quotes:

In [89]:
# import the csv library
import csv

# initialize the top-level array
data_values = []

# open the file and append rows as arrays to the data_values
with open('hit_albums.csv') as csvfile:
    # note that we can interchangably use ' and " in general
    # for the quotechar, however we use ' so that we can use " without escaping
    filereader = csv.reader(csvfile, delimiter=',', quotechar='"')
    # the row here is an array
    for row in filereader:
        print("Row: " + str(row))
        data_values.append(row)

# Store the header in a separate array
header = data_values.pop(0)
   
print()    
print(header)
print()
print(data_values)

Row: ['Artist', 'Album', 'Released', 'Genre', 'Certified sales (millions)', 'Claimed sales (millions)']
Row: ['Michael Jackson', 'Thriller', '1982', 'Pop, rock, R&B', '45.4', '65']
Row: ['AC/DC', 'Back in Black', '1980', 'Hard rock', '25.9', '50']
Row: ['Pink Floyd', 'The Dark Side of the Moon', '1973', 'Progressive rock', '22.7', '45']
Row: ['Whitney Houston / Various artists', 'The Bodyguard', '1992', 'Soundtrack/R&B, soul, pop', '27.4', '44']
Row: ['Meat Loaf', 'Bat Out of Hell', '1977', 'Hard rock, progressive rock', '20.6', '43']
Row: ['Eagles', 'Their Greatest Hits (1971â€“1975)', '1976', 'Rock, soft rock, folk rock', '32.2', '42']
Row: ['Bee Gees / Various artists', 'Saturday Night Fever', '1977', 'Disco', '19', '40']
Row: ['Fleetwood Mac', 'Rumours', '1977', 'Soft rock', '27.9', '40']
Row: ['Shania Twain', 'Come On Over', '1997', 'Country, pop', '29.6', '39']
Row: ['Led Zeppelin', 'Led Zeppelin IV', '1971', 'Hard rock, heavy metal', '29', '37']
Row: ['Michael Jackson', 'Bad', '

To do computation on the numerical dimensions of this table, we need to convert the strings to numbers. If we just do this the simple way, it won't work: 

In [91]:
for row in data_values: 
    row[2] = int(row[2])
    row[4] = float(row[4])
    row[5] = float(row[5])
        
data_values

ValueError: could not convert string to float: ''

This is because the last column, `Claimed sales (millions)` doesn't have values for each row. In that case, the conversion throws the above `ValueError`. 

These errors are also called Exceptions. [Exceptions](https://docs.python.org/3/reference/compound_stmts.html#try) are error states that can be raised and caught:

In [92]:
for row in data_values: 
    # need to try and catch the exception because the column contains NaN values
    try:
        row[2] = int(row[2])
        row[4] = float(row[4])
        row[5] = float(row[5])
    except ValueError: 
        row[5] = None
    
data_values

[['Michael Jackson', 'Thriller', 1982, 'Pop, rock, R&B', 45.4, 65.0],
 ['AC/DC', 'Back in Black', 1980, 'Hard rock', 25.9, 50.0],
 ['Pink Floyd',
  'The Dark Side of the Moon',
  1973,
  'Progressive rock',
  22.7,
  45.0],
 ['Whitney Houston / Various artists',
  'The Bodyguard',
  1992,
  'Soundtrack/R&B, soul, pop',
  27.4,
  44.0],
 ['Meat Loaf',
  'Bat Out of Hell',
  1977,
  'Hard rock, progressive rock',
  20.6,
  43.0],
 ['Eagles',
  'Their Greatest Hits (1971â€“1975)',
  1976,
  'Rock, soft rock, folk rock',
  32.2,
  42.0],
 ['Bee Gees / Various artists',
  'Saturday Night Fever',
  1977,
  'Disco',
  19.0,
  40.0],
 ['Fleetwood Mac', 'Rumours', 1977, 'Soft rock', 27.9, 40.0],
 ['Shania Twain', 'Come On Over', 1997, 'Country, pop', 29.6, 39.0],
 ['Led Zeppelin',
  'Led Zeppelin IV',
  1971,
  'Hard rock, heavy metal',
  29.0,
  37.0],
 ['Michael Jackson', 'Bad', 1987, 'Pop, funk, rock', 20.3, 34.0],
 ['Alanis Morissette',
  'Jagged Little Pill',
  1995,
  'Alternative rock',


So, here we have matrix that we could work with. In reality, we probably would want to structure the data a little differently: Instead of treating each row as an array, we'd want to treat each dimension (column) as an array, as this makes the column homogeneous and it makes it easy to calculate means, etc. 

### 2.4 Reading CSV with Pandas

Now, let's take a look at what it takes to read this file using pandas.

In [93]:
hit_albums = pd.read_csv("hit_albums.csv")
hit_albums

,Artist,Album,Released,Genre,Certified sales (millions),Claimed sales (millions)
0,Michael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65.0
1,AC/DC,Back in Black,1980,Hard rock,25.9,50.0
2,Pink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45.0
3,Whitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44.0
4,Meat Loaf,Bat Out of Hell,1977,"Hard rock, progressive rock",20.6,43.0
...,...,...,...,...,...,...
72,Tracy Chapman,Tracy Chapman,1988,Folk rock,20.0,NaN
73,Usher,Confessions,2004,R&B,20.0,NaN
74,Various artists,Flashdance: Original Soundtrack from the Motio...,1983,Soundtrack,20.0,NaN
75,Whitney Houston,Whitney,1987,"Pop, R&B",20.0,NaN


Well, that was different! 

Pandas provides the insanely powerful ['read_csv()'](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) method. Also see [the documentation](http://pandas.pydata.org/pandas-docs/stable/io.html) for more info on all I/O operations in pandas, including writing CSV files. 

You can pass a lot of arguments to the method, such as delimiter, quote-chars, etc., but for our case the default parameters just worked. 

We've also just created our first data frame! Let's look at data frames in detail next. 

# 3. Data Frames

A data frame is a column-oriented data structure where each column is a pandas series.

Here are two cheat sheets that might prove useful:
[Cheat sheet 1](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
[Cheat sheet 2](http://datacamp-community-prod.s3.amazonaws.com/dbed353d-2757-4617-8206-8767ab379ab3).

We've already loaded a data frame from file, but for completeness sake, let's create one in code: 

In [94]:
bandInfo = pd.DataFrame({
        "Name":["Led Zeppelin", "The Beatles", "Rolling Stones", "Radiohead"],
        "No Members":[4, 4, 4, 5],
        "No Albums":[9, 12, 29 ,9]
    })
bandInfo

,Name,No Members,No Albums
0,Led Zeppelin,4,9
1,The Beatles,4,12
2,Rolling Stones,4,29
3,Radiohead,5,9


This dataframe was initialized with a dictonary of column headers as keys and column data as values.

Just as a series, a data frame has an index, which corresponds to the first column here. In this case the index was automatically generated, but as for the series, we could use explicit values for the index. 

We can access columns in a data frame, which returns a series:

In [95]:
bandInfo["Name"]

0      Led Zeppelin
1       The Beatles
2    Rolling Stones
3         Radiohead
Name: Name, dtype: object

In [96]:
type(bandInfo["Name"])

pandas.core.series.Series

And obviously, we can do all the things we've learned about to this column/series. 

The previous example used columns to create the data frame. We can also create a data frame from rows. This doesn't make a ton of sense in this example, but data could be available like this from your data source, like if you're parsing a CSV.

In [97]:
bandInfo2 = pd.DataFrame([
        {"Name":"Led Zeppelin", "No Albums":9, "No Members":4},
        {"Name":"The Beatles", "No Albums":12, "No Members":4},
        {"Name":"Rolling Stones", "No Albums":29, "No Members":4},
        {"Name":"Radiohead", "No Albums":9, "No Members":5},
    ])
bandInfo2

,Name,No Albums,No Members
0,Led Zeppelin,9,4
1,The Beatles,12,4
2,Rolling Stones,29,4
3,Radiohead,9,5


While a series has only one axis, a dataframe has two, one for the rows (the index or '0' axis), one for the columns (the column or '1' axis). We can check out these axes:

In [98]:
bandInfo.axes

[RangeIndex(start=0, stop=4, step=1),
 Index(['Name', 'No Members', 'No Albums'], dtype='object')]

We can access these axes directly:

In [99]:
# The row axis
bandInfo.axes[0]

RangeIndex(start=0, stop=4, step=1)

In [100]:
# The columns axis
bandInfo.axes[1]

Index(['Name', 'No Members', 'No Albums'], dtype='object')

## Exploring Data Frames

You might have noticed that data frames are rendered in nice HTML tables within Jupyter Notebooks. For small data frames, just showing all the data makes sense, but for larger datasets, like our `hit_albums` dataset, plotting 70+ rows can be annoying, and for datasets with hundreds or thousands of rows it can be prohibitive. By default, a data frame only prints a limited number of elements (notice the `...` in row 30 of the output of `hit_albums` above – only the first 30 and last 30 are printed. 

When working with data, e.g., when transforming or loading a dataset, it is important to see the raw data, for example, to check if a transformation was done correctly. Often, however, it's sufficient to see a part of the data, e.g., the first couple of rows and/or the last couple of rows. We can do this with the `head()` and `tail()` function:

In [101]:
# head shows the first 5 rowsof a datset
hit_albums.head()

,Artist,Album,Released,Genre,Certified sales (millions),Claimed sales (millions)
0,Michael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65.0
1,AC/DC,Back in Black,1980,Hard rock,25.9,50.0
2,Pink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45.0
3,Whitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44.0
4,Meat Loaf,Bat Out of Hell,1977,"Hard rock, progressive rock",20.6,43.0


In [102]:
# we can specify how much to show
hit_albums.head(8)

,Artist,Album,Released,Genre,Certified sales (millions),Claimed sales (millions)
0,Michael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65.0
1,AC/DC,Back in Black,1980,Hard rock,25.9,50.0
2,Pink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45.0
3,Whitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44.0
4,Meat Loaf,Bat Out of Hell,1977,"Hard rock, progressive rock",20.6,43.0
5,Eagles,Their Greatest Hits (1971–1975),1976,"Rock, soft rock, folk rock",32.2,42.0
6,Bee Gees / Various artists,Saturday Night Fever,1977,Disco,19.0,40.0
7,Fleetwood Mac,Rumours,1977,Soft rock,27.9,40.0


In [104]:
# tail shows the last five rows in a datasaet
hit_albums.tail()

,Artist,Album,Released,Genre,Certified sales (millions),Claimed sales (millions)
72,Tracy Chapman,Tracy Chapman,1988,Folk rock,20.0,NaN
73,Usher,Confessions,2004,R&B,20.0,NaN
74,Various artists,Flashdance: Original Soundtrack from the Motio...,1983,Soundtrack,20.0,NaN
75,Whitney Houston,Whitney,1987,"Pop, R&B",20.0,NaN
76,Shakira,Laundry Service,2001,"Pop, Rock",20.0,NaN


We can check out the dimensions of the data frame:

In [103]:
hit_albums.shape

(77, 6)

Here we learn that our dataset has 77 rows and 6 columns.

We can also get more info about the dataset using the info method, which is especially helpful to see the data types of the columns:

In [105]:
hit_albums.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 6 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Artist                      77 non-null     object 
 1   Album                       77 non-null     object 
 2   Released                    77 non-null     int64  
 3   Genre                       77 non-null     object 
 4   Certified sales (millions)  77 non-null     float64
 5   Claimed sales (millions)    31 non-null     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 3.7+ KB


As for series, we can get a rough description of the numerical values of the dataset.

In [107]:
hit_albums.describe()

,Released,Certified sales (millions),Claimed sales (millions)
count,77.000000,77.000000,31.000000
mean,1988.662338,22.409091,35.354839
std,9.949359,4.521620,7.855140
min,1967.000000,13.100000,30.000000
25%,1983.000000,20.000000,30.000000
50%,1987.000000,20.600000,32.000000
75%,1996.000000,25.000000,39.500000
max,2015.000000,45.400000,65.000000


We don't see any descriptions of the columns of non-numerical type. We can, however, get a summary by directly accessing a column:

In [108]:
hit_albums["Artist"].describe()

count                  77
unique                 56
top       Michael Jackson
freq                    5
Name: Artist, dtype: object

Here we can see that Michael Jackson is the top artist in this list, with five albums. Are there other artists with multiple albums in the list? We can answer that question with the value_counts() method:

In [109]:
hit_albums["Artist"].value_counts()

Michael Jackson                      5
Madonna                              3
Shania Twain                         3
The Beatles                          3
Various artists                      3
Bon Jovi                             2
Pink Floyd                           2
Whitney Houston                      2
Backstreet Boys                      2
Eagles                               2
Britney Spears                       2
Celine Dion                          2
Mariah Carey                         2
Adele                                2
Eminem                               1
Oasis                                1
Cyndi Lauper                         1
Boston                               1
TLC                                  1
Ace of Base                          1
Avril Lavigne                        1
Green Day                            1
Eric Clapton                         1
Def Leppard                          1
George Michael                       1
Simon & Garfunkel        

We can look at whether the numerical columns in our data frame are correlated using the [`corr()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html) method. By default, this calculates a Pearson correlation between the column, excluding NaN values. Not surprisingly, we see a rather strong correlation (0.81) between certified and claimed sales. 

In [110]:
hit_albums.corr()

,Released,Certified sales (millions),Claimed sales (millions)
Released,1.000000,-0.052109,-0.320593
Certified sales (millions),-0.052109,1.000000,0.810280
Claimed sales (millions),-0.320593,0.810280,1.000000


We can also do transpose a dataframe:

In [111]:
hit_albums.T

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
Artist,Michael Jackson,AC/DC,Pink Floyd,Whitney Houston / Various artists,Meat Loaf,Eagles,Bee Gees / Various artists,Fleetwood Mac,Shania Twain,Led Zeppelin,...,Prince & the Revolution,Shania Twain,Shania Twain,Supertramp,Tina Turner,Tracy Chapman,Usher,Various artists,Whitney Houston,Shakira
Album,Thriller,Back in Black,The Dark Side of the Moon,The Bodyguard,Bat Out of Hell,Their Greatest Hits (1971–1975),Saturday Night Fever,Rumours,Come On Over,Led Zeppelin IV,...,Purple Rain,The Woman in Me,Up!,Breakfast in America,Private Dancer,Tracy Chapman,Confessions,Flashdance: Original Soundtrack from the Motio...,Whitney,Laundry Service
Released,1982,1980,1973,1992,1977,1976,1977,1977,1997,1971,...,1984,1995,2002,1979,1984,1988,2004,1983,1987,2001
Genre,"Pop, rock, R&B",Hard rock,Progressive rock,"Soundtrack/R&B, soul, pop","Hard rock, progressive rock","Rock, soft rock, folk rock",Disco,Soft rock,"Country, pop","Hard rock, heavy metal",...,"Pop, rock, R&B","Country, pop","Country, pop, world music","Progressive rock, art rock","Pop, rock, R&B",Folk rock,R&B,Soundtrack,"Pop, R&B","Pop, Rock"
Certified sales (millions),45.4,25.9,22.7,27.4,20.6,32.2,19.0,27.9,29.6,29.0,...,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
Claimed sales (millions),65.0,50.0,45.0,44.0,43.0,42.0,40.0,40.0,39.0,37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 3.1 Slicing Data Frames

A common task is to create subsets of a dataframe. Check out the official [user guide for more info on this](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html).  

Column access/slicing works by directly using brackets `[]` on the data frame. Row access/slicing works by using the `.loc[]` indexer. 

We can explicitly define the the **column(s)** we want by their lables:

In [112]:
# a single column
hit_albums["Artist"].head()

0                      Michael Jackson
1                                AC/DC
2                           Pink Floyd
3    Whitney Houston / Various artists
4                            Meat Loaf
Name: Artist, dtype: object

We can use an array of lables if we want multiple columns. 

In [113]:
# specifying multiple columns in an array
hit_albums = hit_albums[["Artist","Certified sales (millions)", "Claimed sales (millions)"]]
hit_albums.head()

,Artist,Certified sales (millions),Claimed sales (millions)
0,Michael Jackson,45.4,65.0
1,AC/DC,25.9,50.0
2,Pink Floyd,22.7,45.0
3,Whitney Houston / Various artists,27.4,44.0
4,Meat Loaf,20.6,43.0


Slicing of columns requires the `iloc` operator

This: 
```python
hit_albums["Artist":"Genre"]
```
Doesn't work.

One break with the convention that rows have to be accessed via `loc` or `iloc` is simple numerical slicing. The documentation claims that's for convenience, since this is so common: 

In [114]:
hit_albums[:3]

,Artist,Certified sales (millions),Claimed sales (millions)
0,Michael Jackson,45.4,65.0
1,AC/DC,25.9,50.0
2,Pink Floyd,22.7,45.0


Using these access methods we can also update the order: 

In [115]:
hit_albums[["Certified sales (millions)", "Claimed sales (millions)", "Artist"]].head()

,Certified sales (millions),Claimed sales (millions),Artist
0,45.4,65.0,Michael Jackson
1,25.9,50.0,AC/DC
2,22.7,45.0,Pink Floyd
3,27.4,44.0,Whitney Houston / Various artists
4,20.6,43.0,Meat Loaf


We can set a column to be the index:

In [117]:
hit_albums_reindexed = hit_albums.set_index("Artist")
hit_albums_reindexed.head()

,Certified sales (millions),Claimed sales (millions)
Artist,,
Michael Jackson,45.4,65.0
AC/DC,25.9,50.0
Pink Floyd,22.7,45.0
Whitney Houston / Various artists,27.4,44.0
Meat Loaf,20.6,43.0


#### Slicing with `loc`

We can retrieve **rows** and columns using the `loc` indexer by name. Depending on whether the label is unique or not, we get either a series or a data frame. 

Generally, `loc` is preferred over direct access via brackets. 

The general syntax is

```
df.loc[rows, columnns]
```
Here rows, columns can be explicit labels, lists of labels, or slicing operators. 

Here's a simple example: 

In [118]:
hit_albums_reindexed.loc["Meat Loaf"]

Certified sales (millions)    20.6
Claimed sales (millions)      43.0
Name: Meat Loaf, dtype: float64

And an example with multiple keys that returns a data frame: 

In [119]:
hit_albums_reindexed.loc["Michael Jackson"]

,Certified sales (millions),Claimed sales (millions)
Artist,,
Michael Jackson,45.4,65.0
Michael Jackson,20.3,34.0
Michael Jackson,17.6,32.0
Michael Jackson,20.0,NaN
Michael Jackson,20.0,NaN


The second argument in the `loc` array can be used to access columns 

In [120]:
hit_albums_reindexed.loc["Michael Jackson", "Certified sales (millions)"]

Artist
Michael Jackson    45.4
Michael Jackson    20.3
Michael Jackson    17.6
Michael Jackson    20.0
Michael Jackson    20.0
Name: Certified sales (millions), dtype: float64

Here is an example with a list of row labels. 

In [121]:
hit_albums_reindexed.loc[["Michael Jackson", "Meat Loaf"], "Certified sales (millions)"]

Artist
Michael Jackson    45.4
Michael Jackson    20.3
Michael Jackson    17.6
Michael Jackson    20.0
Michael Jackson    20.0
Meat Loaf          20.6
Name: Certified sales (millions), dtype: float64

We can use slice operations. Remember, that slicing by label (`loc`) includes the last value, by index (`iloc`) does not. 


Note that this doesn't work if we use labels that have duplicates as indexers. 

In [ ]:
hit_albums_reindexed.loc["Green Day":"Supertramp"]

# this wouldn't work
# hit_albums_reindexed.loc["Green Day":"Michael Jackson"]

This can also be combined with slicing columns: 

In [ ]:
hit_albums_reindexed.loc["Green Day":"Supertramp", ["Certified sales (millions)"]]

We can also slice columns with loc. Let's re-load the full dataset first:  

In [125]:
full_hit_albums = pd.read_csv("hit_albums.csv")
full_hit_albums.head()

,Artist,Album,Released,Genre,Certified sales (millions),Claimed sales (millions)
0,Michael Jackson,Thriller,1982,"Pop, rock, R&B",45.4,65.0
1,AC/DC,Back in Black,1980,Hard rock,25.9,50.0
2,Pink Floyd,The Dark Side of the Moon,1973,Progressive rock,22.7,45.0
3,Whitney Houston / Various artists,The Bodyguard,1992,"Soundtrack/R&B, soul, pop",27.4,44.0
4,Meat Loaf,Bat Out of Hell,1977,"Hard rock, progressive rock",20.6,43.0


This creates a slice containing all rows and the columns from Artist to Released: 

In [126]:
full_hit_albums.loc[:,"Artist":"Released"]

,Artist,Album,Released
0,Michael Jackson,Thriller,1982
1,AC/DC,Back in Black,1980
2,Pink Floyd,The Dark Side of the Moon,1973
3,Whitney Houston / Various artists,The Bodyguard,1992
4,Meat Loaf,Bat Out of Hell,1977
...,...,...,...
72,Tracy Chapman,Tracy Chapman,1988
73,Usher,Confessions,2004
74,Various artists,Flashdance: Original Soundtrack from the Motio...,1983
75,Whitney Houston,Whitney,1987


Here's a slice for the first 10 rows and columns Artists to Released: 

In [127]:
full_hit_albums.loc[:10,"Artist":"Released"]

,Artist,Album,Released
0,Michael Jackson,Thriller,1982
1,AC/DC,Back in Black,1980
2,Pink Floyd,The Dark Side of the Moon,1973
3,Whitney Houston / Various artists,The Bodyguard,1992
4,Meat Loaf,Bat Out of Hell,1977
5,Eagles,Their Greatest Hits (1971–1975),1976
6,Bee Gees / Various artists,Saturday Night Fever,1977
7,Fleetwood Mac,Rumours,1977
8,Shania Twain,Come On Over,1997
9,Led Zeppelin,Led Zeppelin IV,1971


Here is the same thing using `iloc`, i.e., index based slicing:

In [128]:
full_hit_albums.iloc[0:10, 0:3]

,Artist,Album,Released
0,Michael Jackson,Thriller,1982
1,AC/DC,Back in Black,1980
2,Pink Floyd,The Dark Side of the Moon,1973
3,Whitney Houston / Various artists,The Bodyguard,1992
4,Meat Loaf,Bat Out of Hell,1977
5,Eagles,Their Greatest Hits (1971–1975),1976
6,Bee Gees / Various artists,Saturday Night Fever,1977
7,Fleetwood Mac,Rumours,1977
8,Shania Twain,Come On Over,1997
9,Led Zeppelin,Led Zeppelin IV,1971


### 3.2 Broadcasting

Of course, we can use broadcasting and filtering based on boolean masks just as we do for series.

Here we boradcast an operation on a series and set it to a new column. 

In [ ]:
full_hit_albums["Certified sales"] =  full_hit_albums["Certified sales (millions)"] * 1000000
full_hit_albums.head()

Here we filter out all of the albums that were released before 1990. 

In [ ]:
mask = full_hit_albums["Released"] > 1990
mask.head()

In [ ]:
full_hit_albums.loc[mask].head()

Or, short: 

In [ ]:
full_hit_albums.loc[full_hit_albums["Released"] > 2000]

### 3.3 Missing values

Let's take care of some NaN values

In [ ]:
hit_albums.tail()

One way would be to just drop them.

In [ ]:
hit_albums.dropna().tail()

However, that's pretty aggressive here – we're dropping more than half of our dataset. We could also just remove the claimed sales. 

An alternative would be to fill the missing values with our best guess: 
We can use the [`fillna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) on it.

We could replace all NaN values with 0s: 

In [ ]:
hit_albums.fillna(0).tail()

Thought it's probably better to use the forward fill (`ffill`) method here. By default, `ffill` will use the value of the previous row to fill a NaN value: 

In [129]:
hit_albums.fillna(method="ffill").tail()

,Artist,Certified sales (millions),Claimed sales (millions)
72,Tracy Chapman,20.0,30.0
73,Usher,20.0,30.0
74,Various artists,20.0,30.0
75,Whitney Houston,20.0,30.0
76,Shakira,20.0,30.0


It probably makes more sense to use forward fill along the columns, so that the certified sales are filled into the claimed sales if necessary. We can do that by specifying `axis=1` so that `ffill` works on the columns: 

In [130]:
# storing the result here
hit_albums = hit_albums.fillna(axis=1, method='ffill')
hit_albums

,Artist,Certified sales (millions),Claimed sales (millions)
0,Michael Jackson,45.4,65.0
1,AC/DC,25.9,50.0
2,Pink Floyd,22.7,45.0
3,Whitney Houston / Various artists,27.4,44.0
4,Meat Loaf,20.6,43.0
...,...,...,...
72,Tracy Chapman,20.0,20.0
73,Usher,20.0,20.0
74,Various artists,20.0,20.0
75,Whitney Houston,20.0,20.0


### 3.4 Grouping Data Frames

Very often, we want to aggregate data. Given the hit-albums dataset, for example, we might want to ask how many albums each artist in that list has sold in total. We can do these aggregations using the [group-by method](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html). 

We can specify a column to group by, for example, "Artist". We can look at the groups created:

In [134]:
grouped = hit_albums.groupby("Artist")
grouped.groups

{'ABBA': [31], 'AC/DC': [1], 'Ace of Base': [50], 'Adele': [20, 56], 'Alanis Morissette': [11], 'Avril Lavigne': [57], 'Backstreet Boys': [48, 49], 'Bee Gees / Various artists': [6], 'Bob Marley & The Wailers': [37], 'Bon Jovi': [32, 54], 'Boston': [58], 'Britney Spears': [35, 59], 'Bruce Springsteen': [22], 'Carole King': [38], 'Celine Dion': [12, 18], 'Cyndi Lauper': [52], 'Def Leppard': [61], 'Dire Straits': [23], 'Eagles': [5, 14], 'Eminem': [55], 'Eric Clapton': [60], 'Fleetwood Mac': [7], 'George Michael': [62], 'Green Day': [63], 'Guns N' Roses': [30], 'James Horner': [24], 'Led Zeppelin': [9], 'Linkin Park': [36], 'Lionel Richie': [64], 'Madonna': [25, 39, 40], 'Mariah Carey': [15, 41], 'Meat Loaf': [4], 'Metallica': [26], 'Michael Jackson': [0, 10, 16, 65, 66], 'Nirvana': [27], 'Norah Jones': [42], 'Oasis': [53], 'Phil Collins': [43], 'Pink Floyd': [2, 28], 'Prince & the Revolution': [67], 'Queen': [44], 'Santana': [29], 'Shakira': [76], 'Shania Twain': [8, 68, 69], 'Simon & G

Note that the keys map to a set of indices. For example, Michael Jackson's albums are found at indices [0, 10, 16, 65, 66].

Once we have created these groups, we can specify what to do with it.

Pandas has a couple of built in functions to make this easy. For example, we can just call `sum()`:

In [135]:
grouped.sum().head(10)

,Certified sales (millions),Claimed sales (millions)
Artist,,
ABBA,29.0,29.0
AC/DC,25.9,50.0
Ace of Base,23.0,23.0
Adele,42.3,50.0
Alanis Morissette,24.8,33.0
Avril Lavigne,20.0,20.0
Backstreet Boys,48.0,48.0
Bee Gees / Various artists,19.0,40.0
Bob Marley & The Wailers,25.0,25.0


We can see here that we've summed up the data for each column. However, note that NaN plus some number is still NaN, which is treated as 0 here. So let's work with a slice of the dataframe instead. 


Now we sort them, and have a nice result:

In [136]:
grouped.sum().sort_values("Certified sales (millions)", ascending=False).head()

,Certified sales (millions),Claimed sales (millions)
Artist,,
Michael Jackson,123.3,171.0
Shania Twain,69.6,79.0
Madonna,69.4,80.0
Various artists,65.9,80.0
Eagles,53.7,74.0


An alternative is the `count` method. 

In [ ]:
grouped.count().sort_values("Certified sales (millions)", ascending=False).head(10)

 A very generic solution is the `agg()` function, which we can pass a function to do things with the data:

In [ ]:
#here we pass the sum function, which calcualtes the sum of a list, to the group
grouped.agg(sum).head()

We can also pass in numpy functions. 

In [137]:
import numpy as np
grouped.agg(np.max).sort_values("Certified sales (millions)", ascending=False).head(20)

,Certified sales (millions),Claimed sales (millions)
Artist,,
Michael Jackson,45.4,65.0
Eagles,32.2,42.0
Shania Twain,29.6,39.0
ABBA,29.0,29.0
Led Zeppelin,29.0,37.0
Britney Spears,28.0,28.0
Various artists,28.0,32.0
Spice Girls,28.0,28.0
Bon Jovi,28.0,28.0


Here is an aggregation with an in-line function definition where we still create the sum, but also multiply by a million. We use a [lambda expression](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to define the function:

In [138]:
grouped.agg(lambda rows : sum([cell * 1000000 for cell in rows])).head(10)

,Certified sales (millions),Claimed sales (millions)
Artist,,
ABBA,29000000.0,29000000.0
AC/DC,25900000.0,50000000.0
Ace of Base,23000000.0,23000000.0
Adele,42300000.0,50000000.0
Alanis Morissette,24800000.0,33000000.0
Avril Lavigne,20000000.0,20000000.0
Backstreet Boys,48000000.0,48000000.0
Bee Gees / Various artists,19000000.0,40000000.0
Bob Marley & The Wailers,25000000.0,25000000.0


Lambda expressions are just a different way of defining a function in line, without assigning it a name. They only work for a single statement. 

Here is a simple lambda expression, which returns a function, which we assign to the variable add:

In [ ]:
add = lambda a, b : a+b

In [ ]:
add(2, 3)

So, let's take this apart: 

```python
[cell * 1000000 for cell in rows]
```

This part is a list comprehension, that takes an array `rows` and multiplies every element with 1,000,000. 

The surrounding `sum` is a call to the sum function, so adds up the values in the just modified array. 

And finally, the lambda expression packs all of this in a function. 

Here is a different, longer way to write this: 

In [ ]:
def sumUpAndMultiplyByMillion(rows):
    multiplied_rows = [cell * 1000000 for cell in rows]
    summed_value = sum(multiplied_rows)
    return summed_value

grouped.agg(sumUpAndMultiplyByMillion).head(10)

## 4. A note on buit-in Plotting

Dataframes have built-in plotting capabilities based on the [matplotlib](http://matplotlib.org/) library. Refer to the data visualization notebook for all capabilities of plotting with Pandas.

First, we have to import the matplotlib library, and tell Jupyter to display the images directly here:

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# This next line tells jupyter to render the images inline
%matplotlib inline

Then we can simply call the plot attribute:

In [ ]:
hit_albums.plot()

We can also selected certain columns using labelled indexes and then plot.

In [ ]:
hit_albums["Certified sales (millions)"].plot()

We can also use bar-charts instead of line-charts:

In [ ]:
hit_albums[["Certified sales (millions)", "Claimed sales (millions)"]].plot(kind="bar")

The default is a line chart. This doesn't make much sense, since it's mixing index of the row with sales. We're better off plotting only the two different sales figures.

A better way to compare certified and claimed sales is a scatterplot:

In [ ]:
# this is currently broken due to a pandas bug. 
hit_albums.plot.scatter(x="Certified sales (millions)", y="Claimed sales (millions)")

Or a histogram: 

In [ ]:
hit_albums['Certified sales (millions)'].plot.hist(bins=10, alpha=0.5)

Of course there is more sophisticated plotting (please follow the visualization notebook from the lectures)

## Now, you can complete the Bootcamp activity before you move to the clinic